# Investigación para mejorar el resultado del reto Kaggle
## Titanic-DataSet

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Después de pensar que datos seleccionar

### PassengerId - No depende de sobrevivir
### Pclacss - Es necesario para la predicción -Seleccionada
### Name - No es útil 
### Sex - Las mujeres tienen la primera oportunidad -Seleccionada
### Age - La edad depende de la supervivencia -Seleccionada
### SibSp - Tener hermanos/cónyuge depende de la supervivencia -Seleccionada
### Parch - El número de hijos depende de la supervivencia - Seleccionada
### Ticket - El billete no afecta
### Fare - Un billete más caro tiene más posibilidades de sobrevivir -Seleccionada
### Cabin - No afecta a los supervivientes.
### Embarked - Crea un impacto en los supervivientes -Seleccionada


In [3]:
# Nos quedamos con las seleccionadas
seleccion = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
x = df[seleccion]
y = df.Survived

In [4]:
df.Age = df.Age.fillna(df.Age.mean())
df.Embarked = df.Embarked.fillna("S")

In [5]:
# Estuve investigando sobre esta parte del código donde transformo los string en enteros.

# Me da un warning avisando que los cambios los aplico a una copia del DataFrame,
# como es lo que quiero hacer dejo el warning no afecta a mi código.
# Me gustaría saber como evitarlo para dejar un código más limpio pero no encontre solución

# Enlaces de interés:
# https://interactivechaos.com/es/manual/tutorial-de-machine-learning/label-encoding
# https://es.stackoverflow.com/questions/253084/settingwithcopywarning-python-pandas

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x.Sex = encoder.fit_transform(x.Sex)
x.Embarked = encoder.fit_transform(x.Embarked)

/tmp/ipykernel_16518/2038870102.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.Sex = encoder.fit_transform(x.Sex)
/tmp/ipykernel_16518/2038870102.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.Embarked = encoder.fit_transform(x.Embarked)


### Entrenamiento

### Parámetros "train_test_split" usados
#### test_size -->entre 0,0 y 1,0 y representar la proporción del conjunto de datos que se incluirá en la división de prueba.
#### random_state --> Permite establecer la semilla del generador aleatorio(aún no lo entiendo pero investigando vi a muchas personas usar el "42")

In [6]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#### ¿Por qué Random_state=42? 
#### Si no establece random_state a 42, cada vez que ejecute su código de nuevo, generará un conjunto de prueba diferente. Con el tiempo, usted (o su algoritmo de aprendizaje automático) será capaz de ver el conjunto de datos, que desea evitar.

##### https://thecleverprogrammer.com/2020/12/17/why-random_state42-in-machine-learning/

# XGBClassifier algoritmo de predicción

In [18]:
# https://xgboost.readthedocs.io/en/stable/python/python_api.html
# colsample_bylevel: número de variables a sortear por cada nuevo nivel.
# colsample_bytree: número de variables a sortear por cada iteración.
# gamma: Reducción mínima de pérdidas necesaria para realizar una 
# partición adicional en un nodo de la hoja del árbol.
# max_depth: profundidad máxima del árbol.
# min_child_weight: Suma mínima de peso de instancia(hessian) necesaria en un hijo.
# n_estimators: El número de rondas de un modelo 
# nthread: por defecto el número máximo de hilos disponibles si no se establece
# random_state: Semilla de números aleatorios.
from xgboost import XGBClassifier
classifier = XGBClassifier(colsample_bylevel= 0.9,
                    colsample_bytree = 0.9, # Sorteamos el 50 % de las variables
                    gamma=0.9,
                    max_depth= 10,
                    min_child_weight= 1,
                    n_estimators= 10,
                    nthread= 6,
                    random_state= 7)
classifier.fit(x_train,y_train)
classifier.score(x_test,y_test)


0.8547486033519553

#### Despúes de investigar con este algoritmo **XgBoost** mi opinión es mala y buena, dado que si no sabes usar sus parametros, que son muchos, puedes sacar una muestra que se aleja de la verdad consigiendo parametros muy altos sin sentido. Probe a modificar "colsample_by_tree" entre otros parametros y aumentaron mucho el resultado pero no me parece lógico ya que no compredo bien todos los parametros. Es la segunda vez que invesitgo sobre algoritomos de predicción, me gusto mucho y llamó mi atención. 
#### La parte que creo buena del algoritmo es que si lo entiendes en profundidad es un algoritmo muy completo.

In [19]:
test_data = pd.read_csv('test.csv')
test_x = test_data[seleccion]

In [20]:
test_x.Age = test_x.Age.fillna(test_x.Age.median())
test_x.Fare = test_x.Fare.fillna(test_x.Fare.median())

/tmp/ipykernel_16518/3772690814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x.Age = test_x.Age.fillna(test_x.Age.median())
/tmp/ipykernel_16518/3772690814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x.Fare = test_x.Fare.fillna(test_x.Fare.median())


In [21]:
test_x.Sex = encoder.fit_transform(test_x.Sex)
test_x.Embarked = encoder.fit_transform(test_x.Embarked)

/tmp/ipykernel_16518/2222074209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x.Sex = encoder.fit_transform(test_x.Sex)
/tmp/ipykernel_16518/2222074209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x.Embarked = encoder.fit_transform(test_x.Embarked)


In [22]:
prediction = classifier.predict(test_x)

# Guardamos nuestro modelo para enviarlo

In [23]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId,'Survived': prediction})
output.to_csv('submission.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
